In [1]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import tensorflow as tf
from transformers import BertTokenizer
import torch
device = torch.device("cpu")
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

c:\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
df = pd.read_csv('intent_final.csv')
# change column name to intent
df = df.rename(columns={'error': 'intent'})

df.head()


,prompt,intent
0,i'm getting frequent errors when i try to acce...,performance
1,example:i am experiencing slow internet speeds...,performance
2,i'm experiencing slow download speeds on my in...,performance
3,my internet connection has been running extrem...,performance
4,my internet connection suddenly dropped and i ...,performance


In [14]:

possible_labels = df.intent.unique()
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
df['intent_id'] = df.intent.replace(label_dict)
df.head()
# print(df1)
df.to_csv("intent_final.csv", index=False)

In [2]:
enitity_model = tf.keras.models.load_model('enitity_model')

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

# preparing the data that will be passed to the model as an input for testing
def prepare_data(input_text, tokenizer):
    token = tokenizer.encode_plus(
        input_text,
        max_length=32, 
        truncation=True, 
        padding='max_length', 
        add_special_tokens=True,
        return_tensors='tf'
    )
    return {
        'input_ids': tf.cast(token.input_ids, tf.float64),
        'attention_mask': tf.cast(token.attention_mask, tf.float64)
    }

def make_prediction(model, processed_data, classes=['battery', 'printer', 'wifi', 'monitor', 'computer', 'keyboard',
       'mouse']):
    # The predict method is used to pass the processed_data to the neural network
    #  and obtain the predicted class probabilities.
    # The result of the predict method is a 2-dimensional array
    # the first dimension represents the number of samples in processed_data 
    # the second dimension represents the number of classes in the output layer.
    # The indexing [0] is used to extract the predicted class probabilities
    # The np.argmax method is used to extract the index of the class with the highest probability
    # The classes map the index to its corresponding class name
    probs = model.predict(processed_data)[0]
    # returns the class with the highest probability
    return classes[np.argmax(probs)]
     

InternalError: cudaGetDevice() failed. Status: cudaGetErrorString symbol not found.